In [1]:
import requests
import pandas as pd
import time
from operator import itemgetter
import csv
from functools import lru_cache
import logging
from dataclasses import dataclass

In [198]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [220]:
@lru_cache(maxsize=128)
def get_team_matches(team_id):
    while True:
        try:
            team_matches_url = f"https://api.opendota.com/api/teams/{team_id}/matches"
            response = requests.get(team_matches_url)
            response.raise_for_status()
            return response.json()
        
        except (requests.RequestException, ValueError) as e:
            logging.warning(f"Error getting team matches for team_id {team_id}: {e}")
            logging.info("Retrying in 60 seconds...")
            time.sleep(60)


In [210]:
@dataclass
class config:
    key_to_player = ['account_id','win','hero_id']
    keys_to_pick_bans = ['is_pick', 'hero_id', 'team', 'order']

In [211]:
team_id = (7391077,8728920,1061269,8605863,8894818,15,7119388,8254400,7422789,8260983,8291895,8599101,8255756,39,8255888,8597976,8724984,8574561,8244493,2163)
all_matches_data = []

In [212]:
for team in team_id:
    team_matches = get_team_matches(team)
    for match in team_matches:
        match_id = match['match_id']

        radiant_team, dire_team = (team, match['opposing_team_id']) if match['radiant_win'] else (match['opposing_team_id'], team)

        match_data = {
            'match_id': match_id,
            'radian_team': radiant_team,
            'dire_team': dire_team,
            'win': match['radiant_win'],
        }

        all_matches_data.append(match_data)

df = pd.DataFrame(all_matches_data).drop_duplicates(subset='match_id')

csv_filename = 'all_match.csv'
df.to_csv('all_match.csv', index=False)
logging.info(f'Data saved to {csv_filename}')

INFO:root:Data saved to all_match.csv
